In [24]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [25]:
# Adapter Layer

def read_csv_to_df(bucket, key, decoding = 'utf-8', sep = ','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

def write_df_to_s3(bucket, df, key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer, index=False)
    bucket.put_object(Body=out_buffer.getvalue(), Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files

In [26]:
# Application Layer

def extract(bucket, date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket, date)]
    df = pd.concat([read_csv_to_df(bucket, obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df, columns, arg_date):
    df = df.loc[:, columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'], inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date >= arg_date]
    return df

def load(bucket, df, trg_key, trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format
    write_df_to_s3(bucket, df, key)
    return True

def etl_report1(src_bucket, trg_bucket, date_list, columns, arg_date, trg_key, trg_format):
    df = extract(src_bucket, date_list)
    df = transform_report1(df, columns, arg_date)
    load(trg_bucket, df, trg_key, trg_format)
    return True
    

In [27]:
# Application Layer - not core

def return_date_list(bucket, arg_date, src_format, meta_key):
    min_date = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    try:
        df_meta = read_csv_to_df(bucket, meta_key)
        dates = [(min_date + timedelta(days=x)) for x in range(0, (today-min_date).days + 1)]
        src_dates = set(pd.to_datetime(df_meta['source_date']).dt.date)
        dates_missing = set(dates[1:]) - src_dates
        if dates_missing:
            min_date = min(set(dates[1:]) - src_dates) - timedelta(days=1)
            return_dates = [date.strftime(src_format) for date in dates if date >= min_date]
            return_min_date = (min_date + timedelta(days=1)).strftime(src_format)
        else:
            return_dates = []
            return_min_date = datetime(2200, 1, 1).date()
    except bucket.session.client('s3').execptions.NoSuchKey:
        return_dates = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0, (today-min_date).days + 1)]
        return_min_date = arg_date
    return return_min_date, return_dates

In [28]:
# main function entrypoint

def main():
    # Parameters/Configurations
    # Later read config
    arg_date = '2021-05-12'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-1234'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    meta_key = 'meta_file.csv'
    
    # Init
    s3 = boto3.resource('s3')
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    # run application
    extract_date, date_list = return_date_list(bucket_trg, arg_date, src_format, meta_key)
    etl_report1(bucket_src, bucket_trg, date_list, columns, extract_date, trg_key, trg_format)

In [29]:
# run

main()

## Reading the uploaded file

In [30]:
trg_bucket = 'xetra-1234'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
for obj in bucket_trg.objects.all():
    print(obj.key)

meta_file.csv
xetra_daily_report_20210510_101629.parquet
xetra_daily_report_20210510_143346.parquet
xetra_daily_report_20210511_125520.parquet
xetra_daily_report_20210512_084101.parquet
xetra_daily_report_20210514_132039.parquet
xetra_daily_report_20210514_133253.parquet


In [33]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20210514_133253.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [34]:
df_report

ISIN        Date  opening_price_eur  closing_price_eur  \
0     AT00000FACC2  2021-05-14               8.75               8.81   
1     AT0000606306  2021-05-14              19.53              19.61   
2     AT0000609607  2021-05-14              17.00              16.84   
3     AT0000652011  2021-05-14              32.50              32.73   
4     AT0000720008  2021-05-14               7.00               7.08   
...            ...         ...                ...                ...   
2612  XS2265368097  2021-05-14              15.18              15.16   
2613  XS2265369574  2021-05-14              22.57              22.52   
2614  XS2265369731  2021-05-14              10.19              10.17   
2615  XS2265370234  2021-05-14              24.11              23.99   
2616  XS2284324667  2021-05-14              26.41              26.64   

      minimum_price_eur  maximum_price_eur  daily_traded_volume  \
0                  8.75               8.81                  625   
1                 19.50              19.71                 3052   
2                 16.82              17.00                  490   
3                 32.50              32.73                  869   
4                  7.00               7.08                   90   
...                 ...                ...                  ...   
2612              15.16              15.18                    0   
2613              22.52              22.57                    0   
2614              10.17              10.19                    0   
2615              23.99              24.11                   25   
2616              26.41              26.64                    0   

      change_prev_closing_%  
0                      0.23  
1                      0.82  
2                     -1.17  
3                      1.74  
4                      0.28  
...                     ...  
2612                   0.23  
2613                   0.64  
2614                   1.57  
2615                   0.66  
2616                   0.42  

[2617 rows x 8 columns]